In [1]:
import os 
import yaml
def read_config(path):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The API key for the Amadeus Flights API.
    """
    
    # Get the directory of the current script
    script_dir = path

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        configs = yaml.safe_load(stream)
        API_KEY = configs['openai']['api_key']
            
    return API_KEY
path = r"C:\Users\johna\OneDrive\Documents\api_keys"  # Change to the location of your apikeys.yml
API_KEY = read_config(path)

In [2]:
from haystack.nodes import PreProcessor
from haystack.utils import convert_files_to_docs
from haystack.document_stores import FAISSDocumentStore
from sqlalchemy import create_engine

# pre-process docs 
def preprocess_docs(doc_dir):
    all_docs = convert_files_to_docs(dir_path=doc_dir)
    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_respect_sentence_boundary=True,
        split_overlap=30, 
        split_length=100
    )
    docs = preprocessor.process(all_docs)
    print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")
    return docs


# create FAISS
def vector_stores(docs):
    engine = create_engine('sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db')  # change to your local directory
    try:
        # Attempt to drop the table
        engine.execute("DROP TABLE document")
    except Exception as e:
        # Catch any exceptions, likely due to the table not existing
        print(f"Exception occurred while trying to drop the table: {e}")
    
    document_store = FAISSDocumentStore(sql_url='sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db', faiss_index_factory_str="Flat", embedding_dim=768) # change to your local directory
    document_store.write_documents(docs)
    
    return document_store

C:\Users\johna\anaconda3\envs\longfunctioncall_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
doc_dir = r"C:\Users\johna\anaconda3\envs\longfunctioncall_env\long_functioncall\knowledge_base"
docs = preprocess_docs(doc_dir)
document_store = vector_stores(docs)

Preprocessing: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.72docs/s]


n_files_input: 1
n_docs_output: 1176


Writing Documents: 10000it [00:02, 3638.53it/s]                                                                        


In [22]:
import openai
from haystack.nodes.base import BaseComponent
from typing import List
import json

class OpenAIFunctionCall(BaseComponent):
    outgoing_edges = 1

    def run(self, documents: List[str]):
        
        # Try to extract the content and print the first few content strings
        try:
            document_content_list = [doc.content for doc in documents]
            print("documents extracted")
            document_content = " ".join(document_content_list)
        except Exception as e:
            print("Error extracting content:", e)
            return

        print("DOCUMENTS!!", document_content_list)
        functions = [
            {
                "name": "update_dataframe",
                "description": "write the fund details to a dataframe",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "Product_reference_num": {
                            "type": "string",
                            "description": "The FCA product reference number which will be six or seven digits"
                        },
                        "investment_objective": {
                            "type": "string",
                            "description": "You should return the investment objective of the fund. This is likely to be something like this: The Fund aims to grow your investment over t – t + delta t years"
                        },
                        "investment_policy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment policy, no more than two sentences."
                        },
                        "investment_strategy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment strategy, no more than two sentences."
                        },
                        "ESG": {
                            "type": "string",
                            "description": "Return either True, or False. True if the fund is an ESG fund, False otherwise."
                        },
                        "fund_name": {
                            "type": "string",
                            "description": "Return the name of the fund"
                        },
                    },
                },
                "required": ["Product_reference_num", 
                             "investment_objective", 
                             "investment_policy",
                             "investment_strategy",
                             "ESG",
                             "fund_name"]
            }
        ]
        
        openai.api_key = API_KEY
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "system", "content": document_content}],
            functions=functions,
            function_call="auto",  # auto is default, but we'll be explicit
        )

        function_call_args = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])
        
        return function_call_args, "output_1"

    def run_batch(self, documents: List[str]):
        # You can either process multiple documents in a batch here or simply loop over the run method
        results = []
        for document_content in document_content:
            result, _ = self.run(document_content)
            results.append(result)
        return results, "output_1"

In [5]:
# create our pipeline
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-mpnet-base-v2"
)
document_store.update_embeddings(retriever)

C:\Users\johna\anaconda3\envs\longfunctioncall_env\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Batches: 100%|█████████████████████████████████████████████████████████████████████████| 37/37 [03:28<00:00,  5.63s/it]
Documents Processed: 10000 docs [03:28, 47.92 docs/s]                                                                  


In [24]:
p = Pipeline()
p.add_node(component=retriever, name="retriever", inputs=["Query"])
p.add_node(component=OpenAIFunctionCall(), name="OpenAIFunctionCall", inputs=["retriever"])
res = p.run(query="introduction")

Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.95it/s]


documents extracted
DOCUMENTS!! ['On the introduction of any new Fund or Class, a revised Prospectus will be issued\nincluding the relevant details of each such Fund or Class.\n\nInvestment and Borrowing Powers\nThe eligible securities markets and eligible derivatives markets on which the Funds may\ninvest are set out in Appendix 1. A detailed statement of the general investment and\nborrowing restrictions and the extent to which the Company may use derivatives for the\npurposes of investment and Efficient Portfolio Management techniques is set out in Section\n3.\n\n', 'Information on the past performance of each Fund is contained in Appendix 3.\n\nFurther Funds\nSubject to the approval of the FCA, the ACD may establish additional Funds from time to\ntime. On the introduction of any new Fund or Class, a revised Prospectus will be issued\nincluding the relevant details of each such Fund or Class.\n\nInvestment and Borrowing Powers\nThe eligible securities markets and eligible derivative

In [25]:
Product_reference_num = res['Product_reference_num']
investment_objective = res['investment_objective']
investment_strategy = res['investment_strategy']
investment_policy = res['investment_policy']
ESG = res['ESG']
fund_name = res['fund_name']

In [8]:
import pandas as pd

def update_dataframe(prr, investment_objective, investment_strategy, investment_policy, ESG, fund_name):

    # Define the columns
    columns = ['Product_reference_num', 'fund_name', 'investment_objective', 'investment_strategy', 'investment_policy', 'ESG']
    
    # Create an empty DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    
    # Create a dictionary for the new record
    new_record = {
        'Product_reference_num': Product_reference_num,
        'fund_name': fund_name,
        'investment_objective': investment_objective,
        'investment_strategy': investment_strategy,
        'investment_policy': investment_policy,
        'ESG': ESG
    }

    # Check if a record with the given prr already exists
    if Product_reference_num in df['Product_reference_num'].values:
        # Update the existing record
        idx = df[df['Product_reference_num'] == Product_reference_num].index[0]
        df.loc[idx] = new_record
    else:
        # Convert the new record to a DataFrame
        new_df = pd.DataFrame([new_record])
        # Concatenate the existing DataFrame with the new record
        df = pd.concat([df, new_df], ignore_index=True)

    return df

In [26]:
df = update_dataframe(Product_reference_num, investment_objective, investment_strategy, investment_policy, ESG, fund_name)
df

,Product_reference_num,fund_name,investment_objective,investment_strategy,investment_policy,ESG
0,123456,Morgan Stanley Insight Fund,The Fund aims to grow your investment over t –...,The investment process emphasizes a bottom-up ...,The Fund primarily invests in a portfolio of e...,False
